## Imports
---
Before we begin, we import the top-level necessary packages. Moreover, we include the algorithm specific packages from the local directories.

In [ ]:
# Get the system environment variables.
import sys

# General Imports:
import numpy as np
from time import time

# Variational:
from auxiliary import collect_obs
from auxiliary import initialize_Ab0
from core import smoothing

# Plotting:
from matplotlib.pyplot import figure, show, rcParams

%matplotlib inline

# Change the default figure size.
rcParams['figure.figsize'] = (14.0, 7.0)

## Simulation for Lotka-Volterra

In [ ]:
from dynamics import sys_lotka_volterra as dynamics

# Show current python version.
print(" >> Python version is: {0}\n".format(sys.version))

'''
TIME-WINDOW PARAMETERS:
'''

# Initial, final and time step.
t0 = 0; 
tf = 2; 
dt = 0.01

# Define the time-window of inference.
Tw = np.arange(t0, tf + dt, dt)

# Number of discretized points.
N = Tw.shape[0]

'''
SYSTEM SPECIFIC PARAMETERS:
'''

# Dimensionality of the system.
D = 2

# Stochastic Noise (variance).
sigma_Noise = 0.2 * np.eye(D)

# Drift parameters (alpha, beta, delta, gamma).
theta_Drift = np.array([2, 1, 1, 4])

# Observation Noise (variance).
obs_Noise = 0.25 * np.eye(D)

# Initial states
x0 = np.array([5, 3])

# Print a message.
print("System tested: {0}, T=[{1},{2}]\n".format('Lotka-Volterra', t0, tf))

'''
OBSERVATIONS SECTION:
'''

# Create the (artificial) true trajectory.
xt_true = dynamics.system_path(Tw, x0, sigma_Noise, theta_Drift)

# Define the observation density (# of observations per time unit).
n_Obs = 10

# We need at least one observation (per time unit)
n_Obs = np.max([n_Obs, 1])

# Sample the noisy observations from the true path.
obsX, obsY = collect_obs.collect_obs(xt_true, Tw, n_Obs, obs_Noise)

dynamics.plot_sample_path(Tw, xt_true, Tw[obsX], obsY)

# Observation operator: np.eye(D)
H = np.eye(D)

'''
PRIOR MOMEMTS:
'''

# Prior moment of initial condition noise variance.
# p(x0) ~ N(mu, tau0)
tau0 = 0.5 * np.eye(D)

# Get the true sample value at time t=0
prior_x0 = {'mu0': xt_true[0], 'tau0': tau0}

# Initial mean m(t=0)
m0 = xt_true[0] + 0.1 * np.random.randn(1)

# Initial covariance matrix S(t=0): K*np.eye(D)
S0 = 0.25 * np.eye(D)


'''
PACKING PARAMETERS:
'''

# ODE solver: {'Euler', 'Heun', 'RK2', 'RK4'}
ode_method = 'Euler'

# Create a dictionary to hold all the model parameters.
sde_struct = {
    'Sig': sigma_Noise, 
    'theta': theta_Drift, 
    'Rig': obs_Noise,
    'D': D, 
    'H': H, 
    'obsX': obsX,
    'obsY': obsY,
    'px0': prior_x0,
    'Tw': Tw, 
    'dt': dt, 
    'N': N, 
    'ode_method': ode_method,
    'checkGradf': False
}

'''
INITIALIZATION:
'''

# Maximum number of iterations.
nit = 500

# Generate initial variational parameters (initial search point).
Ab0 = initialize_Ab0.initialize_Ab0(S0, sde_struct)

# Main Operation.
print(' [VGPA] (Smoothing) Experiment in progress. Please wait ...')

# Start the timer.
tic = time()

# # Full Variational approximation.
# Fmin, mParam = smoothing.smoothing(dynamics.energy_mode, Ab0, m0, S0, sde_struct, nit)

# # Stop the timer.
# ttime = time() - tic

# # Display termination message.
# print(' [VGPA] (Smoothing) Experiment ended in {0:.2f} seconds.'.format(ttime))

# # Display the minimum free energy.
# print(' Minimum var. free energy is {0:.2f}.\n'.format(Fmin))

# '''
# PLOTTING SECTION:
# '''

In [ ]:
S0.shape